In [38]:
import pandas as pd
import numpy as np
#from pandas_profiling import ProfileReport

In [95]:
municipios = pd.read_excel("Dados/Municipios/lista-1262municipios-semiarido-2017.xlsx")
municipios = municipios.rename(columns=municipios.iloc[1])
municipios = municipios.drop([0, 1]).reset_index(drop=True)
municipios = municipios.drop('#', axis=1).reset_index(drop=True)

In [96]:
idh_tot = pd.read_csv("Dados/Registros_Administrativos/dados_registros_administrativos_total_2012_2027.csv")
#idh_tot = idh_tot[(idh_tot['IBGE7'].isin(municipios['Cod IBGE']))]
idh_leg = pd.read_csv("Dados/Registros_Administrativos/dados_registros_administrativos_LEGENDA.csv")

In [97]:
censo = pd.read_csv("Dados/Censo/Censo_municipal_estadual_nacional-MUN91-00-10.csv")
censo = censo[(censo['Codmun7'].isin(municipios['Cod IBGE']))]
#censo = censo.drop(censo[censo['IDHM'] > 0.4].index).reset_index(drop=True)
censo_siglas = pd.read_csv("Dados/Censo/Censo_municipal_estadual_nacional-Siglas.csv")

In [98]:
alagoas = pd.read_csv("Dados/Dados IBGE/Alagoas.csv")
bahia = pd.read_csv("Dados/Dados IBGE/Bahia.csv")
ceara = pd.read_csv("Dados/Dados IBGE/Ceara.csv")
maranhao = pd.read_csv("Dados/Dados IBGE/Maranhao.csv")
minas_gerais = pd.read_csv("Dados/Dados IBGE/Minas Gerais.csv")
paraiba = pd.read_csv("Dados/Dados IBGE/Paraiba.csv")
pernambuco = pd.read_csv("Dados/Dados IBGE/Pernambuco.csv")
piaui = pd.read_csv("Dados/Dados IBGE/Piaui.csv")
rio_grande_do_norte = pd.read_csv("Dados/Dados IBGE/Rio Grande do Norte.csv")
sergipe = pd.read_csv("Dados/Dados IBGE/Sergipe.csv")

dados_ibge = pd.concat([alagoas, bahia, ceara, maranhao, minas_gerais, paraiba, pernambuco, piaui, rio_grande_do_norte, sergipe], ignore_index=True)

In [100]:
censo_to_merge = censo.drop_duplicates(subset='Codmun7', keep='last')
idh_tot_to_merge = idh_tot.drop_duplicates(subset='IBGE7', keep='last')
censo_idh_merged = pd.merge(idh_tot_to_merge[['IBGE7', 'NOME', 'POP_TOT']], censo_to_merge[['Codmun7', 'IDHM']], left_on='IBGE7', right_on='Codmun7', how='inner')
df = censo_idh_merged.drop('Codmun7', axis=1)
df = df.sort_values(by='IBGE7')

In [101]:
censo_sorted = censo.sort_values(by=['ANO', 'Codmun7'])
censo_sorted['Crescimento_IDHM'] = censo_sorted.groupby('Codmun7')['IDHM'].pct_change()
censo_sorted = censo_sorted.dropna().reset_index(drop=True)
censo_sorted = censo_sorted.sort_values(by='Codmun7')
df['Crescimento_IDHM'] = censo_sorted['CRESCIMENTO_IDHM']

In [104]:
dados_ibge = dados_ibge.sort_values(by='Munic&iacute;pio [-]')
df['AREA_KM²'] = dados_ibge['&Aacute;rea Territorial - km&sup2; [2022]']
df['POP/KM²'] = df['POP_TOT']/df['Area (Km²)']
df['bonk'] = dados_ibge['Densidade demogr&aacute;fica - hab/km&sup2; [2022]']

In [105]:
df[censo_idh_merged['POP_TOT'] >= 0].sort_values(by='IDHM')

,IBGE7,NOME,POP_TOT,IDHM,Crescimento_IDHM,Area (Km²),POP/KM²,bonk
840,2703304,Inhapi,18652,0.484,0.722420,37050,0.503428,97.5
151,2209658,São Francisco de Assis do Piauí,5657,0.485,1.012448,13622,0.415284,96.3
1016,2916500,Itapicuru,36211,0.486,0.562701,38915,0.930515,98.4
769,2609154,Manari,21047,0.487,0.650847,10223,2.058789,95.8
49,2202653,Caxingó,5330,0.488,0.827715,9534,0.559052,97.4
...,...,...,...,...,...,...,...,...
438,2408003,Mossoró,295619,0.720,0.208054,20952,14.109345,99.0
555,2504009,Campina Grande,410332,0.720,0.198003,17632,23.272005,97.9
917,2903201,Barreiras,157638,0.721,0.260490,30805,5.117286,98.3
1237,3151206,Pirapora,56706,0.731,0.190554,32244,1.758653,98.5


In [ ]:
cod_municipios = [260280, 260700, 230810, 270840]
last_values = censo[~censo['Codmun6'].isin(cod_municipios)].sort_values(by='ANO').sort_values(by=['Município']).groupby('Codmun6').last().reset_index()
first_values = censo[censo['Codmun6'].isin(cod_municipios)].sort_values(by='ANO').sort_values(by=['Município']).groupby('Codmun6').first().reset_index()

new_censo = pd.concat([first_values, last_values])
new_censo.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = new_censo.copy()
X.drop(columns=['Município', 'UF', 'ANO', 'Codmun6', 'Codmun7'], inplace=True)
X.fillna(0, inplace=True)
X = scaler.fit_transform(X)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=13)
kmeans.fit(X)
new_censo['Classe'] = kmeans.labels_

In [ ]:
new_censo['Classe'].value_counts()

In [ ]:
new_censo[(new_censo['Codmun6'].isin([260280, 260700, 230810, 270840]))].sort_values(by='ANO').sort_values(by=['Classe'])

In [ ]:
new_censo[(new_censo['Classe'].isin([8]))].sort_values(by='ANO').sort_values(by=['IDHM'])